In [1]:
# Install all the required libraries
%pip install pandas
%pip install scikit-learn
%pip install request
%pip install matplotlib
%pip install numpy
%pip install joblib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Read the csv file
df = pd.read_csv('datasets/weather.csv')

In [3]:
# Show quantity of each weather
print(df['Weather'].value_counts())

Weather
Mainly Clear                               2106
Mostly Cloudy                              2069
Cloudy                                     1728
Clear                                      1326
Snow                                        390
Rain                                        306
Rain Showers                                188
Fog                                         150
Rain,Fog                                    116
Drizzle,Fog                                  80
Snow Showers                                 60
Drizzle                                      41
Snow,Fog                                     37
Snow,Blowing Snow                            19
Rain,Snow                                    18
Thunderstorms,Rain Showers                   16
Haze                                         16
Drizzle,Snow,Fog                             15
Freezing Rain                                14
Freezing Drizzle,Snow                        11
Freezing Drizzle                

In [4]:
# show a resumen of dataframe
df.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [5]:
# delete the column 'Date/Time'
df = df.drop(columns=['Date/Time'])


In [6]:
# Rename the columns
df = df.rename(columns={'Temp_C': 'Temp', 
                        'Dew Point Temp_C': 'DewPoint', 
                        'Rel Hum_%': 'Humidity', 
                        'Wind Speed_km/h': 'WindSpeed', 
                        'Visibility_km': 'Visibility', 
                        'Press_kPa': 'Pressure'})

# show columns
df.columns


Index(['Temp', 'DewPoint', 'Humidity', 'WindSpeed', 'Visibility', 'Pressure',
       'Weather'],
      dtype='object')

In [17]:
from pprint import pprint

# Get the max and min of each column
ls = ['Temp', 'DewPoint', 'Humidity', 'WindSpeed', 'Visibility', 'Pressure']
data = {}

for i in ls:
    max = df[i].max()
    min = df[i].min()

    data[i] = {
        'max': max,
        'min': min
    }

pprint(data)

{'DewPoint': {'max': 24.4, 'min': -28.5},
 'Humidity': {'max': 100, 'min': 18},
 'Pressure': {'max': 103.65, 'min': 97.52},
 'Temp': {'max': 33.0, 'min': -23.3},
 'Visibility': {'max': 48.3, 'min': 0.2},
 'WindSpeed': {'max': 83, 'min': 0}}


In [7]:
# assign the values of the variables X and Y
X = df.drop('Weather', axis=1).values

Y = df['Weather'].values

In [8]:
# split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=42)


In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

# Create a model and train it with logistic regression
from sklearn.linear_model import LogisticRegression

# Create a pipeline
p = make_pipeline(SimpleImputer(strategy='mean'),
                  StandardScaler(),
                  MinMaxScaler(), 
                  LogisticRegression(max_iter=1000, n_jobs=3, verbose=0, solver='lbfgs'))

# Train the model
p.fit(X_train, Y_train)



Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('minmaxscaler', MinMaxScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, n_jobs=3))])

In [10]:
# Make predictions
Y_pred = p.predict([X_test[0]])


# Show the prediction
print(Y_pred, Y_test[0])

['Cloudy'] Mostly Cloudy


In [11]:
# check accuracy of model
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score

print("Accuracy Score",accuracy_score(Y_test, p.predict(X_test)))

print("Confusion Matrix: ",confusion_matrix(Y_test, p.predict(X_test)))

print("F1 Score:", f1_score(Y_test, p.predict(X_test), average='weighted'))


Accuracy Score 0.37564029595902104
Confusion Matrix:  [[ 86  37   0 ...   0   0   0]
 [ 51 140   0 ...   0   0   0]
 [  0   6   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   1   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
F1 Score: 0.35626990459124097


In [18]:
# grid search

from sklearn.model_selection import GridSearchCV


# Create a pipeline
p = make_pipeline(SimpleImputer(strategy='mean'),
                  StandardScaler(),
                  MinMaxScaler(), 
                  LogisticRegression(max_iter=1000, n_jobs=3, verbose=0, solver='lbfgs'))

# Create a dictionary with the hyperparameters

param_grid = {
    'logisticregression__C': [0.1, 1, 10, 100],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__solver': ['liblinear', 'lbfgs']
}

# Create a grid search

cv = GridSearchCV(p, param_grid, n_jobs=3, verbose=0, cv=5)
cv.fit(X_train, Y_train)

# Show the best hyperparameters
pprint(cv.best_params_)

/Users/mac/anaconda3/envs/ds-env/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/mac/anaconda3/envs/ds-env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  warnings.warn(
/Users/mac/anaconda3/envs/ds-env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  warnings.warn(
/Users/mac/anaconda3/envs/ds-env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  warnings.warn(
/Users/mac/anaconda3/envs/ds-env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1211: UserWarnin

{'logisticregression__C': 10,
 'logisticregression__penalty': 'l2',
 'logisticregression__solver': 'lbfgs'}


In [20]:
# use the best model
model = LogisticRegression(C=10, penalty='l2', solver='lbfgs', max_iter=1000, n_jobs=3, verbose=0)
p = make_pipeline(SimpleImputer(strategy='mean'),
                  StandardScaler(),
                  MinMaxScaler(), 
                  model)

p.fit(X_train, Y_train)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('minmaxscaler', MinMaxScaler()),
                ('logisticregression',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=3))])

In [22]:
# export the model
import joblib

joblib.dump(p, 'weather.pkl')

['weather.pkl']